## Схема проекта

**Принципиальные моменты:**
*   У страховой компании нет времени "на подумать", решение о приеме на страхование или отказе, а также о тарифе должено быть сделано непосредственно в момент обращения клиента за полисом. Соответственно, сервис должен работать в режиме реального времени и давать ответ в течение максимум нескольких секунд.
*   Сервис скоринга строится обычно отдельным блоком, а фронт-система взаимодействует с сервисом через API (Application programming interface).
*   Запрос данных из внешних источников может производиться как на уровне фронт-системы, так и на уровне сервиса.
*   Один из возможных вариантов - Flask.
*   Передача информации производится путем обмена json-файлами.
*   Для тестирования можно использовать Postman.

  ![](https://drive.google.com/uc?export=view&id=1OAOF1M2U14UJWDmeJg2mwo-pgSwyNyzc)

## JSON

JSON (JavaScript Object Notation) - простой текстовый формат обмена данными, он основан на подмножестве языка программирования JavaScript.<br/>
Например, строка из нашего датасета выглядела бы следующим образом:<br/>
```
{
"ID": 1,
"Exposure": 0.583,
"RecordBeg": "2004-06-01",
"RecordEnd": "",
"DrivAge": 55,
"Gender": "Female",
...
}
```

## При внедрении

**При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данные, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование, запись котировок.

**Особенности:**
*   Библиотека H2O использует виртуальную Java-машину:
    *   ее нужно инициализировать один раз, а не поднимать заново для каждого расчета;
    *   вручную выделить под нее отдельный порт и указать размер используемой памяти;
    *   не создавать каждый раз заново H2O-Frame, а записывать в единожды подготовленный.
*   Необходимо удостовериться, что на всех этапах сервис отрабатывает корректно; например, можно иметь заготовленный массив котировок с заранее известными ответами.
*   Необходимо провести нагрузочное тестирование и удостовериться, что сервис справляется с нагрузкой.
*   Для согласованности версий Python, Java при переносе на другие серверы имеет смысл использовать докеры.



## Flask

Тут будет сервис для обработки запросов на Flask

In [2]:
# Подключение к Google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Google Colab предоставляет виртуальную машину, поэтому мы не можем получить доступ к локальному хосту, как это делаем на нашем локальном компьютере при запуске локального веб-сервера. Что мы можем сделать, так это предоставить его общедоступному URL-адресу с помощью ngrok.

https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b

In [3]:
!pip install flask-ngrok

In [4]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [4]:
# Пробный запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ade7-34-125-174-13.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/Nov/2021 23:13:35] "GET /a HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 23:13:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/Nov/2021 23:13:39] "GET /a HTTP/1.1" 200 -


In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,roc_curve,scorer
import dill
dill._dill._reverse_typemap['ClassType'] = type

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


### **Создаем сервис для обработки запросов к модели**

In [6]:
path2='/content/drive/MyDrive/Colab_Notebooks/ML_in_business/COLLAB_PROJ/My_project_ML_API_Collab/'

In [6]:
!ls drive/MyDrive/Colab_Notebooks/ML_in_business/COLLAB_PROJ

Lesson_9.ipynb		  Step1_Train.ipynb		X_test.csv
logreg_pipeline.dill	  Step2_predict.ipynb		y_test.csv
My_project_ML_API_Collab  Step_3_запрос_к_модели.ipynb


In [7]:
# Загружаем обученные модели
with open(path2+'randomforest_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator FeatureUnion from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid result

In [8]:
X_test = pd.read_csv(path2+"X_test.csv")
y_test = pd.read_csv(path2+"y_test.csv")

In [9]:
X_test.columns

Index(['AccountWeeks', 'ContractRenewal', 'DataPlan', 'DataUsage',
       'CustServCalls', 'DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee',
       'RoamMins'],
      dtype='object')

Запустить сервис и не глушить его, пока работаем 

In [10]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/predict', methods=['GET', 'POST'])
def predict():
  data = {"success": False}
  # ensure an image was properly uploaded to our endpoint
  if request.method == "POST":
      #description, company_profile, benefits = "", "", "qq"

      accountweeks, contractrenewal, dataplan, datausage, custservcalls, daymins, daycalls, monthlycharge, overagefee, roammins = "", "", "", "", "", "", "", "", "", ""
      request_json = request.get_json()

  if request_json['AccountWeeks']:
			accountweeks = request_json['AccountWeeks']
  if request_json['ContractRenewal']:
			contractrenewal = request_json['ContractRenewal']
  if request_json['DataPlan']:
			dataplan = request_json['DataPlan']
  if request_json['DataUsage']:
			datausage = request_json['DataUsage']
      
  if request_json['CustServCalls']:
			custservcalls = request_json['CustServCalls']
  if request_json['DayMins']:
			daymins = request_json['DayMins']
  if request_json['DayCalls']:
			daycalls = request_json['DayCalls']
  if request_json['MonthlyCharge']:
			monthlycharge = request_json['MonthlyCharge']
  if request_json['OverageFee']:
			overagefee = request_json['OverageFee']
  if request_json['RoamMins']:
			roammins = request_json['RoamMins']




  #print(description )  
  df = pd.DataFrame({"AccountWeeks": [accountweeks],
													  "ContractRenewal": [contractrenewal],
													  "DataPlan": [dataplan],
													  "DataUsage": [datausage],
													  "CustServCalls": [custservcalls],
													  "DayMins": [daymins],
													  "DayCalls": [daycalls],
													  "MonthlyCharge": [monthlycharge],
													  "OverageFee": [overagefee],
													  "RoamMins": [roammins]
													})
  #print(df)
  df = df[['AccountWeeks', 'ContractRenewal', 'DataPlan', 'DataUsage', 'CustServCalls', 'DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee', 'RoamMins']].astype(float)
  preds = model.predict_proba(df)
  
  
  
  data["predictions"] = preds[:, 1][0]
  #data["description"] = description
	# indicate that the request was a success
  data["success"] = True

	# return the data dictionary as a JSON response
  return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6400-35-221-47-149.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Nov/2021 21:36:16] "POST /predict HTTP/1.1" 200 -
[2021-11-06 21:41:50,078] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-10-eb7ac776eb2a

In [14]:
for c in cat_feats:
    
    X_train[c] = X_train[c].astype('category')


item_features.columns = [col.lower() for col in item_features.columns]

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('AccountWeeks',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  NumberSelector(key='AccountWeeks')),
                                                                 ('pow_2',
                                                                  NumericPower(key='AccountWeeks',
                                                                               p=4)),
                                                                 ('Scale',
                                                                  StandardScaler(copy=True,
                                                                                 with_mean=True,
                                                                 